Maxwell solver for PEC bodies (EFIE)
=============================
**keys**: Maxwell single layer potential, EFIE, indirect ansatz, MoM, PEC scattering

In [ ]:
import sys
sys.path.append("../build/")
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

We consider a perfect conducter $\Omega \subset \mathbb R^3$ and a plane wave $\boldsymbol E_{\mathrm{inc}} $ with tangential trace $\gamma_R \boldsymbol E_{\mathrm{inc}} = -\boldsymbol m \in \boldsymbol H^{-\frac12}\left( \mathrm{curl}_\Gamma, \Gamma\right)\,.$ The incoming wave thus induces a scattered electric field $\boldsymbol E$ which propagates into $\Omega^c$. The scattered electric field solves the following boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_R \,\boldsymbol E &=& \boldsymbol m, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 

and a possible representation reads

$$ \boldsymbol E(x) = \underbrace{ \kappa \, \int_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}\sigma_y + \frac{1}{\kappa} \nabla \int_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{SL} (\boldsymbol j) } } \,.$$ 

The density $\boldsymbol j$ is determined by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \begin{array}{c} \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): \quad \left\langle \,\mathrm{SL} (\boldsymbol j),\, \boldsymbol v \right\rangle_{-\frac12} = \left\langle \boldsymbol m , \boldsymbol v\right\rangle_{-\frac12} \,. \end{array}$$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM) and the resulting equation for $\boldsymbol j$ is called the **EFIE**, the **electric field integral equation**. 

Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
hull = Box((0,0,0), (100,10,5))
deck1 = Box((30,0,5), (50,10,10))
deck2 = Box((70,0,5), (85,10,15))

shape = hull + deck1 + deck2
mesh = Mesh(OCCGeometry(shape).GenerateMesh (maxh=2.9))
#mesh = Mesh("ship3.stl")
#mesh.Curve(1)
Draw(mesh)
fesL2 = SurfaceL2(mesh,order=0)
print(fesL2.ndof)

Next, we create finite element spaces $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ and $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$:

In [ ]:
order = 4
fesHDiv = HDivSurface(mesh, order=order, complex=True)
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) trial space for Neumann data ( nx curlE ) and test space for BIE
print ("ndof HDiv = ", fesHDiv.ndof)

Define the incoming plane wave and compute the given Dirichlet data $\boldsymbol m$: 

In [ ]:
kappa = 7 # norm
E_inc = CF((-0.66, 0.436, 0.112)) * exp( -1j * (-0.785 * x - 0.453 * y - 0.524 * z))
n = specialcf.normal(3)
m = - Cross( Cross(n, E_inc), n) # m = (nxE)xn in H(curl_Gamma)
gfm = GridFunction(fesHCurl)
gfm.Set(E_inc, definedon=mesh.Boundaries(".*"))
Draw(gfm, mesh, draw_vol=False, order=order, min=-0.5, max=0.5, animate_complex=True);
Draw(gfm, mesh, draw_vol=False, order=order, min=-0.5, max=0.5, animate_complex=True);

Compute the right hand side vector:

In [ ]:
rhs = LinearForm(m * vHDiv.Trace() *ds(bonus_intorder=3)).Assemble()  # <H(curl_Gamma), H(div_Gamma)> 

The discretisation of the above variational formulation leads to a system of linear equations, ie

$$ V\, \mathrm j = \mathrm{rhs}\,,$$

where the single layer operator $V$ on the left is a regular and symmetric matrix.

In [ ]:
j = GridFunction(fesHDiv)
pre = BilinearForm(uHDiv.Trace() * vHDiv.Trace() * ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, intorder=16, leafsize=200, eta=3., eps=1e-8)
    GMRes(A = V.mat, pre=pre, b = rhs.vec, x=j.vec, tol=1e-8, maxsteps=5000, printrates=True)

In [ ]:
Draw (Norm(j), mesh, draw_vol=False, order=3);

In [ ]:
Draw( j, mesh, draw_vol=False, order=3, min=-2, max=2, animate_complex=True);